### Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.

### Design a pipeline that includes the following steps"
1. Use an automated feature selection method to identify the important features in the dataset.
2. Create a numerical pipeline that includes the following steps.

    a. Impute the missing values in the numerical columns using the mean of the column values.
    
    b. Scale the numerical columns using standardisation.
    
3. Create a categorical pipeline that includes the following steps.

    a. Impute the missing values in the categorical columns using the most frequent value of the column.
    
    b. One-hot encode the categorical columns.
4. Combine the numerical and categorical pipelines using a ColumnTransformer.
5. Use a Random Forest Classifier to build the final model.
6. Evaluate the accuracy of the model on the test dataset.

### Note: Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements forthe pipeline.

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
df=sns.load_dataset("iris")

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [11]:
encoder=LabelEncoder()
df["species"]=encoder.fit_transform(df["species"])

In [12]:
numeric_cols=["sepal_length", "sepal_width", "petal_length", "petal_width"]
categorical_cols=[]

In [13]:
#Feature Engg Automation
num_pipeline=Pipeline(steps=[("imputer",SimpleImputer(strategy="mean")),     #Handle missing values
                             ("scaler",StandardScaler())        #Standardization
                            ])
cat_pipeline=Pipeline(steps=["imputer",SimpleImputer(strategy="most_frequent"),   #Handle missing values
                            ("onehotencoder", OneHotEncoder())      #Cat features to numerical
                            ])
preprocessor=ColumnTransformer([("num_pipeline",num_pipeline,numeric_cols),
                               ("cat_pipeline", cat_pipeline, categorical_cols)
                               ])

In [ ]:
#Instead of below steps, we can create a final pipline and train, predict, find accuracy on this pipeline

# Create the final pipeline

#pipeline = Pipeline([
#    ('feature_selection', SelectKBest(k=10)), 
#    ('preprocessor', preprocessor),
#    ('classifier', RandomForestClassifier())
#])

In [14]:
# Independent & Dependent features
X=df.drop("species", axis=1)
y=df["species"]

# Train test split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [19]:
# Automate Model train process
models={
    "Random Forest":RandomForestClassifier(),
    "Decision Tree":DecisionTreeClassifier(),
    "SVM":SVC()
}

def evaluate_model(X_train,y_train,X_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        
        #Train model
        model.fit(X_train,y_train)
        
        #Predict test data
        y_test_pred=model.predict(X_test)
        
        #Get accuracy
        test_model_score=accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
    return report
        
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 1.0, 'Decision Tree': 1.0, 'SVM': 1.0}

All the 3 classifiers give an accuracy of 100%. This is due to non-complexity nature of the data.

### Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [22]:
# Load voting classifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

# Create the Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Create the Logistic Regression Classifier
lr_classifier = LogisticRegression()

# Create the Voting Classifier
voting_classifier=VotingClassifier(estimators=[("rf",rf_classifier),("lr",lr_classifier)],voting="hard")

# Fit the pipeline on the training data
voting_classifier.fit(X_train, y_train)

# Evaluate the accuracy of the model on the test data
y_pred = voting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)

Accuracy: 1.0


Here also, we will get the accuracy of 100% as individual models are also giving the same accuracy due to the same reson.